# Plot results

In [ ]:
import os

# Set parameters
project_root = os.path.realpath(os.pardir)
data_set = 'pixel'
stimulus_sets = ['jitter', 'static']  # 'static'
start_trial = 1
num_trials = 5
lambdas = [4, 8, 16, 32, 64]

epochs = 20
data_augmentation = 0
fresh_data = 0
n_gpus = 1

In [ ]:
# Load accuracy scores and plot
%matplotlib inline
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns


# data_set = 'pixel'
stimulus_set = 'jitter'  # 'static'  # 'jitter'
noise_types = ['Original', 'Salt-and-pepper', 'Additive', 'Single-pixel']  # 'Original'
test_conditions = ['Same', 'Diff', 'NoPix']
results_dir = os.path.join(project_root, 'results', data_set, stimulus_set)

rows = []
test_rows = []

# for stimulus_set in stimulus_sets:
for trial in range(1, 1+num_trials):
    for noise_type in noise_types:
        for lambd in lambdas:

            trial_label = f"{trial}_lambd={lambd}"
            model_name = f"{noise_type}_{trial_label}"

            acc_scores = np.load(os.path.join(results_dir, f'{model_name}_ACC.npy'))
            valacc_scores = np.load(os.path.join(results_dir, f'{model_name}_VALACC.npy'))
            loss = np.load(os.path.join(results_dir, f'{model_name}_LOSS.npy'))
            valloss = np.load(os.path.join(results_dir, f'{model_name}_VALLOSS.npy'))

            with open(os.path.join(results_dir, f'{model_name}_CONDVALACC.json'), "r") as jf:
                cond_acc = json.load(jf)
            with open(os.path.join(results_dir, f'{model_name}_CONDVALLOSS.json'), "r") as jf:
                cond_loss = json.load(jf)

            for condition in test_conditions:
                test_rows.append({'Trial': trial, 'Noise Type': noise_type,
                                 'Condition': condition, 'Lambda': lambd,
                                  'Loss': cond_loss[condition], 'Accuracy': cond_acc[condition]})
            for epoch in range(epochs):
                rows.append({'Trial': trial, 'Noise Type': noise_type, 'Lambda': lambd, 
                             'Evaluation': 'Testing', 'Epoch': epoch+1, 'Loss': valloss[epoch], 
                             'Accuracy': valacc_scores[epoch]})

                rows.append({'Trial': trial, 'Noise Type': noise_type, 'Lambda': lambd, 
                             'Evaluation': 'Training', 'Epoch': epoch+1, 'Loss': loss[epoch], 
                             'Accuracy': acc_scores[epoch]})

scores = pd.DataFrame(rows, columns=['Trial', 'Noise Type', 'Evaluation', 'Lambda', 'Epoch', 'Loss', 'Accuracy'])
test_scores = pd.DataFrame(test_rows, columns=['Trial', 'Noise Type', 'Condition', 'Lambda', 'Loss', 'Accuracy'])
# scores
# test_scores

In [ ]:
g = sns.catplot(x="Condition", y="Accuracy", row='Lambda', col="Noise Type", kind="bar", data=test_scores)